In [1]:
import pyspark
import os

os.environ['SPARK_DIST_CLASSPATH']="""/opt/spark2/conf/yarn-conf:/opt/hadoop2/share/hadoop/common/lib/*:/opt/hadoop2/share/hadoop/common/*:/opt/hadoop2/share/hadoop/hdfs:/opt/hadoop2/share/hadoop/hdfs/lib/*:/opt/hadoop2/share/hadoop/hdfs/*:/opt/hadoop2/share/hadoop/yarn/lib/*:/opt/hadoop2/share/hadoop/yarn/*:/opt/hadoop2/share/hadoop/mapreduce/lib/*:/opt/hadoop2/share/hadoop/mapreduce/*"""

from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc

In [2]:
sc = SparkContext()

In [3]:
ssc = StreamingContext(sc, 10 )
sqlContext = SQLContext(sc)

In [4]:
socket_stream = ssc.socketTextStream("127.0.0.1", 9999)

In [5]:
socket_stream.pprint()

In [ ]:
ssc.start()

In [8]:
ssc.stop(False)

In [5]:
# устанавливаем окно в 60 секунд
lines = socket_stream.window(60)

In [6]:
# словарь для определения имен

from collections import namedtuple

fields = ("hashtag", "count" )
Tweet = namedtuple( 'Tweet', fields )

In [7]:
( lines.flatMap(lambda text: text.split(" ")) #разделяем список
  .filter(lambda word: word.lower().startswith("#")) # ищем хэш-код
  .map(lambda word: (word.lower(), 1)) # все в нижний регистра
  .reduceByKey(lambda a, b: a + b) # складываем 
  .map(lambda rec: Tweet(rec[0], rec[1])) # сохраняем в объект
  .foreachRDD(lambda rdd: rdd.toDF().sort(desc("count")) # в DF
  .limit(10).createOrReplaceTempView("tweets")) ) # регистрация таблички в DF

In [16]:
# должен быть True
"tweets" in sqlContext.tableNames()

False

In [8]:
import time
from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns
import pandas
%matplotlib inline

In [9]:
ssc.start()

In [ ]:
count = 0
while count < 3:
    
    time.sleep(5)
    top_10_tags = sqlContext.sql( 'Select hashtag, count from tweets' )
    top_10_df = top_10_tags.toPandas()
    display.clear_output(wait=True)
    plt.figure( figsize = ( 10, 8 ) )
    sns.barplot( x="count", y="hashtag", data=top_10_df)
    plt.show()
    count = count + 1
    print(conut)

In [31]:
ssc.stop()